In [2]:
import pyvisa

# ─────── Configure VISA ─────────
rm = pyvisa.ResourceManager()
inst = rm.open_resource("TCPIP0::192.168.1.155::5025::SOCKET")  # replace with your scope’s address

# give plenty of time and buffer for large blocks
inst.timeout    = 5000        # 5000 ms before timing out
inst.chunk_size = 10**6       # 1 MB

# ensure line endings are just '\n' so we see exactly what the scope sends
inst.write_termination = '\n'
inst.read_termination  = '\n'

# ─────── Milestone 1 Steps ───────

# 1. Disable the ASCII header so the first byte you read is '#'
inst.write("CHDR OFF")  

# 2. Tell the scope: “Send me every sample, no sparsing, from the first point onward”
inst.write("WFSU SP,0,NP,0,FP,0")  

# 3. Ask for the full waveform block (DAT2 = header + data + end-flag)
inst.write("C1:WF? DAT2")  

# 4. Manually read the IEEE-488.2 block:
#    a) First two bytes: '#' + a digit N telling how many length-digits follow
prefix      = inst.read_bytes(2)              # e.g. b'#9'
num_digits  = prefix[1] - ord('0')            # ASCII '9'→ 9

#    b) Next N bytes: the ASCII decimal length of the data payload
length_str  = inst.read_bytes(num_digits)     # e.g. b'000000070'
data_len    = int(length_str.decode('ascii'))

#    c) Then exactly data_len bytes of waveform payload
payload     = inst.read_bytes(data_len)

#    d) Finally the two-byte terminator (usually b'\n\n')
terminator  = inst.read_bytes(2)

# Combine everything into one raw block
raw_block = prefix + length_str + payload + terminator

# 5. Print the raw bytes as an ASCII-escaped string for easy comparison
print(repr(raw_block))

# Clean up
inst.close()
rm.close()


ValueError: invalid literal for int() with base 10: 'T2,#9000014000TTT'